In [1]:
!pip install l0bnb
from l0bnb import fit_path

In [23]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from statsmodels.api import OLS
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from functools import partial
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP.models import \
     (Stepwise,
      sklearn_selected,
      sklearn_selection_path,
      summarize)

# 8

a. Create a random number generator and use its normal() method
to generate a predictor X of length n = 100, as well as a noise
vector " of length n = 100.

In [14]:
random_numbers = np.random.normal(size=(100,100))
random_numbers.shape

(100, 100)

b. Generate a response vector Y of length n = 100 according to the model where β0, β1, β2, and β3 are constants of your choice.

In [15]:
Y = np.arange(0,100)
B0 = 2
B1 = 3
B2 = 4
B3 = 5
Y.shape

(100,)

c. Use forward stepwise selection in order to select a model contain-
ing the predictors X, X2, . . . , X10. What is the model obtained
according to Cp? Report the coefficients of the model obtained.

In [16]:
def nCp(sigma2, estimator, X, Y):
    "Negative Cp statistic"
    n, p = X.shape
    Yhat = estimator.predict(X)
    RSS = np.sum((Y - Yhat)**2)
    return -(RSS + 2 * p * sigma2) / n 


In [17]:
X = random_numbers
sigma2 = OLS(Y,X).fit().scale

c:\Users\aster\anaconda3\envs\islp2\Lib\site-packages\statsmodels\regression\linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


In [18]:
neg_Cp = partial(nCp, sigma2)

In [12]:
strategy = Stepwise.first_peak(X,
                               direction='forward',
                               max_terms=len(X))

TypeError: [ 1.8188994  -0.02866924  0.44787359 -1.35082944 -1.18072738 -1.01366148
  1.49136351 -0.5794826  -0.58456255 -0.539502   -0.69086736  1.38813736
 -0.45827455 -0.11722291 -1.15807974 -0.29680603 -0.30285284  1.05287145
  0.58270247  0.77921432 -0.22002595  0.82145964  0.77819466  0.87737885
  1.51379396 -0.23933813 -0.80923009  0.79465793 -0.20928729 -0.52059261
  1.53662472 -1.08469475  1.04462602 -1.03225869 -2.3511283   0.48091438
  2.5324273   1.72314504  0.25880566  0.86858704 -1.35895013  0.22907826
  0.5012341  -0.95999372 -0.21767378  0.8299539  -0.10696457  0.8369371
  0.20637094  0.26244145 -1.39322079  1.08874847  0.36553509 -0.74223856
  1.17769551 -1.11419675  0.50844889 -1.72183585 -0.93302645 -0.46341037
 -0.84190279 -1.70018252 -0.97522345  0.64298301 -1.29107014  0.25616927
  1.62918848  1.60913063  0.17628439 -0.63945469  0.64567953 -0.78666094
  1.0522562   0.70492968 -0.5062496   0.82389067  0.15170791 -0.37232825
 -0.51436359 -0.93174333 -0.52081442 -2.0

# 9

a. Split the data set into a training set and a test set.

In [35]:
from sklearn.model_selection import train_test_split

islpCollege = pd.DataFrame(load_data("College"))

train, test = train_test_split(islpCollege, test_size=0.3, random_state=1)
islpCollege

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,No,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
773,Yes,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
774,Yes,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49
775,Yes,10705,2453,1317,95,99,5217,83,19840,6510,630,2115,96,96,5.8,49,40386,99


b. Fit a linear model using least squares on the training set, and
report the test error obtained.

In [34]:
import statsmodels.api as sm
X = islpCollege.drop(columns=["Apps"])
y = islpCollege["Apps"]
print(type(X))
print(type(y))
model = sm.OLS(y, X)
# results = model.fit()

# summarize(results)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).